In [4]:
import sys
import os
from env.env1D import env1D
import filepath as file
import random #nur zum testen
import neat
import pybullet as p
import visualize
import graphviz
import time
import math
import numpy as np
#genomes, config

fitnessDevelopment = []
bestGenomes = []
topFitness = 0
collisionDevelopment = []

        
def cartToSpher(x,y,z):
    z = z-0.36
    r = math.sqrt(x*x+y*y+z*z)
    a = z/r
    theta = math.acos(a)
    phi = math.atan2(y,x)
    pol = [r,theta,phi]
    return pol

def spherToCart(rad, theta, phi):
    x = math.sin(theta) * math.cos(phi) * rad
    y = math.sin(theta) * math.sin(phi) * rad
    z = math.cos(theta) * rad + 0.36
    cart = [x,y,z]
    return cart


def calcDistance(firstPos, secondPos):
    dist = math.sqrt(((firstPos[0]-secondPos[0])**2)+((firstPos[1]-secondPos[1])**2)+((firstPos[2]-secondPos[2])**2))
    return dist

        


def getStartPos(num):
    pos = []
    rows = num/2
    for i in range(num):
        pos.append([3*(i%10), (i/10)*3, 0])
    return pos
        
        
        


In [5]:

numOfTests = 1000
posFile = 'goalPositions'
numberAxis = 3


    
local_dir = file.getPath()
config_path = os.path.join(local_dir, 'config.txt')
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error
global pop
config = neat.config.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path
)
#node_names = {-1:'Position Achse 1', -2: 'Position Achse 2', -3: 'Position Achse 4', -4: 'Distanz r', -5: 'Distanz Theta', -6: 'Distanz Phi', 0:'Achse A1', 1:'Achse A2', 2:'Achse A4'}
#node_names = {-1:'Position Achse 1', -2: 'Position Achse 2', -3: 'Position Achse 4', -4: 'Distanz X', -5: 'Distanz Y', -6: 'Distanz Z', 0:'Achse A1', 1:'Achse A2', 2:'Achse A4'}
#node_names = {-1:'Pos Achse 1', -2: 'Zielwinkel', 0:'Achse 1'}
#visualize.draw_net(config, winner, True, node_names=node_names)


collisionCount = 0
maxStep = 400
step = 0
stepCount = 0
collisionCount = 0
over = 0
under = 0
u5cm = 0
u10cm = 0
u50cm = 0
distances = []
with open(posFile, "rb") as f:
    g = pickle.load(f)
f.close()

goalKart = g
goalSpher = []
for g in goalKart:
    goalSpher.append(cartToSpher(g[0], g[1], g[2]))
    

p.connect(p.GUI)
bot = botEnv(numAxis = numberAxis)
time.sleep(1)
net = neat.nn.FeedForwardNetwork.create(winner,config)

while(step < numOfTests):
    while(stepCount<maxStep):
        if not bot.collisionFlag:
            endeffektor = bot.getBotHeadPos() #pos kartesische Koordinaten [0] = x, [1] = y, [2] = x
            endeffektorSpher = bot.getBotSpherical() #pos räumliche Polarkoordinaten [0] = r, [1] = theta, [2] = phi
            endeffektorOrientation = list(p.getEulerFromQuaternion(p.getLinkState(bot.botId,bot.numJoints-1)[1])) #[0] = roll, [1] = pitch, [2] = yaw
            
 #######################################################################################################################################################               
                #HIER DAS NETZ AUF DIE VERWENDETEN INPUTS ANPASSEN
            output = net.activate((
                                p.getJointState(bot.botId, 0)[0],
                                p.getJointState(bot.botId, 1)[0],
                                p.getJointState(bot.botId, 3)[0],
                                endeffektorSpher[0]-goalSpher[step][0],
                                endeffektorSpher[1]-goalSpher[step][1],
                                endeffektroSpher[2]-goalSpher[step][2]
                                ))
            
 #######################################################################################################################################################           
            
            if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.botId)) > 0:   #Kollision Roboter mit sich selbst
                bot.collisionFlag = True
            if len(p.getContactPoints(bodyA=bot.botId, bodyB=bot.planeId)) > 0: #Kollision Roboter Bodenplatte
                bot.collisionFlag = True
            bot.setMovement(output)
            bot.moveStep()            
        p.stepSimulation()
        time.sleep(1/240)
        stepCount += 1
    
    if not bot.collisionFlag:
        distance = calcDistance(bot.getBotHeadPos(), goalKart[step])
        distances.append(distance)
        if distance < 0.01:
            under += 1
        elif distance < 0.05:
            u5cm += 1
        elif distance < 0.1:
            u10cm += 1
        if distance > 0.1:
            over += 1
    else:
        bot = env1D()
        collisionCount += 1
    distances.append(distance)
    print('Step: ', step, ' distance: ', distance)
    print('Pos:', goalKart[step])
    stepCount = 0
    step += 1
    print(bot.moveDir)
    
print('Points without Collision: ', step)
print('Greates Distance: ', max(distances), ' at Step: ', distances.index(max(distances)))
distanceTotal = 0
for d in distances:
    distanceTotal += d
print('Average Distance: ', distanceTotal/numOfTests)
print('Kollisionen: ', collisionCount)
print('< 1cm :', under)
print('< 5cm :', u5cm)
print('< 10cm :', u10cm)
print('< 50cm: ', u50cm)
print('> 50cm :', over)
    
    
    
    

Key: 8026
Fitness: 99.53299587660855
Nodes:
	0 DefaultNodeGene(key=0, bias=0.04471374602360195, response=1.0, activation=sigmoid, aggregation=sum)
	1 DefaultNodeGene(key=1, bias=0.006431352580811345, response=1.0, activation=sigmoid, aggregation=sum)
	2 DefaultNodeGene(key=2, bias=0.008796376849703774, response=1.0, activation=sigmoid, aggregation=sum)
	1080 DefaultNodeGene(key=1080, bias=0.9385209031770706, response=1.0, activation=sigmoid, aggregation=sum)
	1143 DefaultNodeGene(key=1143, bias=-1.9311499357542277, response=1.0, activation=sigmoid, aggregation=sum)
	1206 DefaultNodeGene(key=1206, bias=0.08704388690946724, response=1.0, activation=sigmoid, aggregation=sum)
	1294 DefaultNodeGene(key=1294, bias=3.3751868091648807, response=1.0, activation=sigmoid, aggregation=sum)
	1299 DefaultNodeGene(key=1299, bias=-1.21950788073551, response=1.0, activation=sigmoid, aggregation=sum)
	1352 DefaultNodeGene(key=1352, bias=-1.7514271540523687, response=1.0, activation=sigmoid, aggregation=

In [ ]:
#TO DO: 
#1.Multiple Robots in the same simulation, write in .py File, maybe problems with Jupyter Lab and p.connect(p.Direct) DONE
#2.Rework the fitness function DONE
#3.Play around with the config file
#4.Collision Detection DONE
#

In [3]:
import pickle
def getGoalPosition(upperRad, lowerRad, upperTheta, lowerTheta, upperPhi, lowerPhi, num): #Minimum Radius = 0.465, Höhe Gelenk = 0.36, MaxRadius = 0.921
    goalPos = []
    goalSpher = []
    for i in range(num):
        rad = random.uniform(lowerRad, upperRad)
        theta = random.uniform(lowerTheta, upperTheta)
        phi = random.uniform(lowerPhi, upperPhi)
        zOffset = 0.36 #Höhe des Gelenkes
        x = math.sin(math.radians(theta)) * math.cos(math.radians(phi)) * rad
        y = math.sin(math.radians(theta)) * math.sin(math.radians(phi)) * rad
        z = math.cos(math.radians(theta)) * rad + zOffset
        goal = [x,y,z]
        goalS = [rad, math.radians(theta), math.radians(phi)]
        goalPos.append(goal)
        goalSpher.append(goalS)
    print(goalPos)
    print(goalSpher)
    return goalPos, goalSpher

goalKart, goalKugel = getGoalPosition(0.91, 0.47, 109, 4, 169, -169, 1000)

with open('Pos3K', 'wb') as KV:
    pickle.dump(goalKart, KV)
KV.close()
with open('Pos3R', 'wb') as P:
    pickle.dump(goalKugel, P)


[[-0.00982293460481691, -0.06594117070793086, 0.9186897701868456], [-0.1511137826557709, 0.5826957166645971, 0.8983040051603914], [0.7201317207148749, 0.38276682451717114, 0.6922447619861954], [-0.15609105960507838, -0.41094023762790444, 1.081597745910544], [0.25445071098562866, -0.10328162069519413, 0.8328635665738318], [0.15530740797545095, 0.39993019471604846, 1.0561305947786992], [0.1501350993649917, -0.07606084366117867, 1.0658465218808213], [-0.200754807341104, 0.3349096673880733, 0.7633082093174581], [-0.3021533710874716, -0.7832085496018766, 0.6036600923863216], [0.12905393932109535, 0.23130314065265947, 1.0691021835233996], [0.32409261189278954, 0.4963809889932789, 0.4790724142384837], [-0.6320816671803282, -0.3663718732325708, 0.21172717174635444], [0.5200222879365873, -0.30343522161195197, 0.9873948922468408], [0.029216388538600417, -0.12218130295209718, 0.9616740304823997], [0.1484414338045367, 0.22438327524935336, 1.0527268892501347], [0.31785129452396094, 0.05259386892449